In [1]:
# Setup
import numpy as np
import os
import symforce

symforce.set_symbolic_api("symengine")
symforce.set_log_level("warning")

# Set epsilon to a symbol for safe code generation.  For more information, see the Epsilon tutorial:
# https://symforce.org/tutorials/epsilon_tutorial.html
symforce.set_epsilon_to_symbol()

from symforce import codegen
from symforce.codegen import codegen_util
from symforce import ops
import symforce.symbolic as sf
from symforce.values import Values
from symforce.notebook_util import display, display_code, display_code_file

In [48]:
import sympy as sp
sp.init_printing()

## Rotation matrices

In [108]:
# Sympy matrices
# Rotate about x-axis

alpha, beta, gamma = sp.symbols("alpha, beta, gamma")

g_I = sp.symbols("g")

# force vector in body fixed frame
f_B = sf.Vector3.symbolic("f_B")

# gravity vector in inertial frame
g_I = sf.V3([0,0,g])

R_x = sf.Matrix(
    [
        [1, 0, 0],
        [0, sf.cos(gamma), -sf.sin(gamma)],
        [0, sf.sin(gamma), sf.cos(gamma)],
    ]
)

R_y = sf.Matrix(
    [
        [sf.cos(beta), 0, sf.sin(beta)],
        [0            , 1,             0],
        [-sf.sin(beta), 0, sf.cos(beta)],
    ]
)

R_z = sf.Matrix(
    [
        [sf.cos(alpha), -sf.sin(alpha), 0],
        [sf.sin(alpha),  sf.cos(alpha), 0],
        [0            ,  0            , 1],
    ]
)

# Rotation matrix from body fixed frame to intertial frame
R_IB = R_z*R_y*R_x

# translational equations of motion in the intertial frame
f_I = (R_IB * f_B) + g_I

# display(R_IB)
# display(f_B)
display(f_I)

⎡cos(α)⋅cos(β)  -sin(α)⋅cos(γ) + sin(β)⋅sin(γ)⋅cos(α)  sin(α)⋅sin(γ) + sin(β)⋅
⎢                                                                             
⎢sin(α)⋅cos(β)  sin(α)⋅sin(β)⋅sin(γ) + cos(α)⋅cos(γ)   sin(α)⋅sin(β)⋅cos(γ) - 
⎢                                                                             
⎣   -sin(β)                 sin(γ)⋅cos(β)                         cos(β)⋅cos(γ

cos(α)⋅cos(γ)⎤
             ⎥
sin(γ)⋅cos(α)⎥
             ⎥
)            ⎦

⎡f_B0⎤
⎢    ⎥
⎢f_B1⎥
⎢    ⎥
⎣f_B2⎦

⎡f_B0⋅cos(α)⋅cos(β) + f_B1⋅(-sin(α)⋅cos(γ) + sin(β)⋅sin(γ)⋅cos(α)) + f_B2⋅(sin
⎢                                                                             
⎢f_B0⋅sin(α)⋅cos(β) + f_B1⋅(sin(α)⋅sin(β)⋅sin(γ) + cos(α)⋅cos(γ)) + f_B2⋅(sin(
⎢                                                                             
⎣                          -f_B0⋅sin(β) + f_B1⋅sin(γ)⋅cos(β) + f_B2⋅cos(β)⋅cos

(α)⋅sin(γ) + sin(β)⋅cos(α)⋅cos(γ))⎤
                                  ⎥
α)⋅sin(β)⋅cos(γ) - sin(γ)⋅cos(α)) ⎥
                                  ⎥
(γ) + g                           ⎦

In [153]:
def R_IB(pose_B: sf.Pose3,
    alpha:sf.Scalar, beta:sf.Scalar, gamma:sf.Scalar, epsilon: sf.Scalar = 0
) -> sf.Vector3:
    """
    Transform a nav point into azimuth / elevation angles in the
    camera frame.

    Args:
        nav_T_cam (sf.Pose3): camera pose in the world
        nav_t_point (sf.Matrix): nav point
        epsilon (Scalar): small number to avoid singularities

    Returns:
        sf.Matrix: (azimuth, elevation)
    """
    
    p_B = pose_B.position()
    R_B = pose_B.rotation()
    

    R_x = sf.Matrix(
        [
            [1, 0, 0],
            [0, sf.cos(gamma), -sf.sin(gamma)],
            [0, sf.sin(gamma), sf.cos(gamma)],
        ]
    )

    R_y = sf.Matrix(
        [
            [sf.cos(beta), 0, sf.sin(beta)],
            [0            , 1,             0],
            [-sf.sin(beta), 0, sf.cos(beta)],
        ]
    )

    R_z = sf.Matrix(
        [
            [sf.cos(alpha), -sf.sin(alpha), 0],
            [sf.sin(alpha),  sf.cos(alpha), 0],
            [0            ,  0            , 1],
        ]
    )
    
    R_IB = R_z*R_y*R_x
    
    # create a Rot3 type from the rotation matrix
    R_IB = sf.Rot3.from_rotation_matrix(R_IB)
    
    # second derivative
    p_B_ddot = p_B
    
    return p_B_ddot

## Create code generators for C++ and python

In [154]:
az_el_codegen_cpp = codegen.Codegen.function(
    func=R_IB,
    config=codegen.CppConfig(),
)

az_el_codegen_python = codegen.Codegen.function(
    func=R_IB,
    config=codegen.PythonConfig(),
)

## Generate C++ code

In [155]:
# generate a C++ function from the python function
az_el_codegen_data = az_el_codegen_cpp.generate_function()

# print("Files generated in {}:\n".format(az_el_codegen_data.output_dir))
# for f in az_el_codegen_data.generated_files:
#     print("  |- {}".format(os.path.relpath(f, az_el_codegen_data.output_dir)))

# show the generated code
display_code_file(az_el_codegen_data.generated_files[0], "C++")

## Generate python code

In [86]:
# # generate a C++ function from the python function
# az_el_codegen_data = az_el_codegen_python.generate_function()

# # print("Files generated in {}:\n".format(az_el_codegen_data.output_dir))
# # for f in az_el_codegen_data.generated_files:
# #     print("  |- {}".format(os.path.relpath(f, az_el_codegen_data.output_dir)))

# # show the generated code
# display_code_file(az_el_codegen_data.generated_files[0], "python")

# Pose

(x,y,z) position + Quaternion for orientation

In [139]:
# Create a pose type from a rotation and translation

# Orientation of body frame wrt world frame
R_B = sf.Rot3.symbolic("R")

# Position of body frame wrt world frame written in the world frame
p_B = sf.Vector3.symbolic("t")

pose_B = sf.Pose3(R=R_B, t=p_B)
display(world_T_body)

display(pose_B.position())

<Pose3 R=<Rot3 <Q xyzw=[R_x, R_y, R_z, R_w]>>, t=(t0, t1, t2)>

⎡t₀⎤
⎢  ⎥
⎢t₁⎥
⎢  ⎥
⎣t₂⎦

# Numerical Evaluation

In [132]:
rot_sym = sf.Rot3.symbolic("rot")
rotN = sf.Rot3.from_storage([1,0,0,0])
rot_num = rot_sym.subs(rot_sym, rotN)

display(rot_num.evalf())  # Numerical evaluation

<Rot3 <Q xyzw=[1.0, 0.0, 0.0, 0.0]>>

In [147]:
p_B = pose_B.position()
type(p_B)

symforce.geo.matrix.Matrix31